# Single qubit gate fidelity

In [1]:
import numpy as np

def singleQubitGateFidelity(T1,T2,gate_length):
    return 1 - 0.5*(1.-2./3.*np.exp(-gate_length/T2)-1./3.*np.exp(-gate_length/T1))
#0.5*(1.-2./3.*np.exp(-gate_length/T2)-1./3.*np.exp(-gate_length/T1)) from Dave

def singleQubitGateFidelity_GaussinDecay(T1,T2,gate_length):
    return 1 - 0.5*(1.-2./3.*np.exp(-(gate_length/T2)**2)-1./3.*np.exp(-gate_length/T1))


In [2]:
singleQubitGateFidelity(18, 2, 0.020)

0.9964981955737693

In [3]:
1-singleQubitGateFidelity_GaussinDecay(18, 2, 0.02)

0.00021841400934241584

In [4]:
# Gate fid vs gate length
%matplotlib
import matplotlib.pyplot as plt

T1, T2 = 40, 45
T1_1, T2_1 = 18,  5

gate_lengths = np.linspace(0, 0.1, 101)

fid = singleQubitGateFidelity(T1, T2, gate_lengths)
fid1 = singleQubitGateFidelity(T1_1, T2_1, gate_lengths)

label_FFT = 'FFT:T1={:.1f},T2={:.1f}'.format(T1, T2)
label_CSFQ = 'CSFQ:T1={:.1f},T2={:.1f}'.format(T1_1, T2_1)
plt.plot(gate_lengths, fid, '.-', label=label_FFT)
plt.plot(gate_lengths, fid1, '.-', label=label_CSFQ)
plt.xlabel('Gate length (us)')
plt.ylabel('Gate Fidelity')
plt.grid()
plt.legend()

Using matplotlib backend: TkAgg


In [7]:
# Gate fid vs T2
%matplotlib
import matplotlib.pyplot as plt

T1 = 18
T2 = np.linspace(2, 16, 1001) # in us
gate_length = 0.02
fid = singleQubitGateFidelity(T1, T2, gate_length)

plt.plot(T2, fid)
plt.xlabel('T2 (us)')
plt.ylabel('Gate Fidelity')
# plt.xscale('log')
# plt.yscale('log')
plt.grid()

Using matplotlib backend: TkAgg


# Two qubit gate fidelity calculation: function defined

In [5]:
import numpy as np


def twoQubitGateFidelity(T1_t, T2_t, T1_c, T2_c, gate_length):
    """ Return the two qubit gate fidelity in percent """
    T1 = [T1_t, T1_c]
    T2 = [T2_t, T2_c]
    T1factor, T2factor = 0, 0

    for i in range(2):
        T1factor += 1. / 15. * np.exp(-gate_length / T1[i])
        T2factor += 2. / 15. * \
            (np.exp(-gate_length /
                    T2[i]) + np.exp(-gate_length * (1. / T2[i] + 1. / T1[1 - i])))

    T1factor += 1. / 15. * np.exp(-gate_length * (1. / T1[0] + 1. / T1[1]))
    T2factor += 4. / 15. * np.exp(-gate_length * (1. / T2[0] + 1. / T2[1]))

    return (1 - 0.75 * (1. - T1factor - T2factor)) * 100

In [6]:
# Function test
T1_t, T2_t, T1_c, T2_c, gate_length = 40, 31, 20, 16, 0.16
# T1_t, T2_t, T1_c, T2_c, gate_length = 20,16, 40, 31, 0.2

fidelity = twoQubitGateFidelity(T1_t, T2_t, T1_c, T2_c, gate_length)
print('fidelity = {:.2f} %'.format(fidelity))

fidelity = 99.16 %


# Fidelity decay function for two-qubit

In [7]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np

num_cl_gates = np.linspace(1, 200, 100)
gate_errors = [0.001, 0.002,0.004,0.006,0.008, 0.01]
# e = 3/4*(1-0.9^(1/1.5))
for gate_error in gate_errors:
    p = (1 - gate_error *4/3)**1.5
    fid = 0.5 + 0.5 *p**num_cl_gates
    plt.plot(num_cl_gates, fid, label=f'e={gate_error:.2g}')
plt.grid('on')
plt.legend()

Using matplotlib backend: TkAgg


# FFT and CSFQ (IBM384QB56)

In [11]:
# Gate fid vs gate length
%matplotlib
import matplotlib.pyplot as plt

T1_t, T2_t, T1_c, T2_c =  40, 45, 18, 5
T1_t, T2_t, T1_c, T2_c =  100, 100, 100, 100
T1_t, T2_t, T1_c, T2_c =  50, 50, 50, 50

gate_lengths = np.linspace(10,300,100) / 1e3

print('T1(FFT), T2(FFT) = {:.3g}, {:.3g} us'.format(T1_t, T2_t))
print('T1(CSFQ), T2(CSFQ) = {:.3g}, {:.3g}\n'.format(T1_c, T2_c))

# for gate_length in gate_lengths:
#     fidelity = twoQubitGateFidelity(T1_t, T2_t, T1_c, T2_c, gate_length)
# #     print('gate length ={:5g} ns, fidelity = {:.2f} %'.format(gate_length*1e3,fidelity))

# gate_lengths = np.linspace(10, 500, 100)/1e3
fid = twoQubitGateFidelity(T1_t, T2_t, T1_c, T2_c, gate_lengths)
plt.plot(gate_lengths, fid)
# plt.plot(gate_lengths, fid2, 'b-')
plt.xlabel('Gate length (us)')
plt.ylabel('Gate Fidelity (%)')
plt.grid()

Using matplotlib backend: TkAgg
T1(FFT), T2(FFT) = 50, 50 us
T1(CSFQ), T2(CSFQ) = 50, 50



In [18]:
%matplotlib
import matplotlib.pyplot as plt

gate_lengths = np.linspace(160,450,100) / 1e3

T1_t, T2_t, T1_c, T2_c =  40, 45, 18, 16
fid1 = twoQubitGateFidelity(T1_t, T2_t, T1_c, T2_c, gate_lengths)
T1_t, T2_t, T1_c, T2_c =  40, 45, 18, 6
fid2 = twoQubitGateFidelity(T1_t, T2_t, T1_c, T2_c, gate_lengths)
T1_t, T2_t, T1_c, T2_c =  40, 45, 18, 3
fid3 = twoQubitGateFidelity(T1_t, T2_t, T1_c, T2_c, gate_lengths)

plt.plot(gate_lengths, fid1)
plt.plot(gate_lengths, fid2)
plt.plot(gate_lengths, fid3)
plt.xlabel('Gate length (us)')
plt.ylabel('Gate Fidelity (%)')
plt.grid()

Using matplotlib backend: TkAgg


In [7]:
plt.title('T1_t, T2_t, T1_c, T2_c = 40, 45, 18, (16,6,3)')

Text(0.5,1,'T1_t, T2_t, T1_c, T2_c = 40, 45, 18, (16,6,3)')

In [57]:
file = 'Z:\Experiments\Rack3\Qlab\experiments\muWaveDetection\JKscripts\gatelengthvsfid1'
np.savetxt(file, gatelengthvsfid)

In [25]:
# Gate fid vs T2
%matplotlib
import matplotlib.pyplot as plt

T1_t, T2_t, T1_c  =  40, 45, 18
T2_c = np.linspace(2, 15, 1001)

gate_length_200 = 0.200
gate_length_300 = 0.300
gate_length_440 = 0.440

fid_200ns = [twoQubitGateFidelity(T1_t, T2_t, T1_c, T2, gate_length_200) for T2 in T2_c]
fid_300ns = [twoQubitGateFidelity(T1_t, T2_t, T1_c, T2, gate_length_300) for T2 in T2_c]
fid_440ns = [twoQubitGateFidelity(T1_t, T2_t, T1_c, T2, gate_length_440) for T2 in T2_c]
    
plt.plot(T2_c, fid_200ns, label='200ns')
plt.plot(T2_c, fid_300ns, label='300ns')
plt.plot(T2_c, fid_440ns, label='440ns')

plt.xlabel('T2_c (us)')
plt.ylabel('Gate Fidelity (%)')
plt.grid()
plt.ylim([93,99.5])
plt.title('T1_t, T2_t, T1_c, gate_length = {:3g}, {:3g}, {:3g} us, {:3g} us'.format(T1_t, T2_t, T1_c, gate_length))
plt.legend()

Using matplotlib backend: Qt5Agg


In [19]:
T1_t, T2_t, T1_c  =  39, 40, 18.6
T2_c = np.linspace(1, 20, 1001)
gate_length = 0.3
fid = [twoQubitGateFidelity(T1_t, T2_t, T1_c, T2, gate_length) for T2 in T2_c]
    
plt.plot(T2_c, fid)

96.79035661988155